# Ensemble Weight Optimization

**Goal**: Find optimal MLP/LightGBM weights for the ensemble.

**Current best**: MLP 0.6, LightGBM 0.4 (CV 0.009004 from exp_013)

**Test variations**:
1. MLP 0.7, LightGBM 0.3 (more MLP emphasis - since [32,16] is best LB)
2. MLP 0.5, LightGBM 0.5 (equal weighting)
3. MLP 0.8, LightGBM 0.2 (heavy MLP emphasis)

**Hypothesis**: Since [32,16] MLP has the best LB (0.0932), giving it more weight might improve generalization.

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


In [2]:
# Data loading
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT NAME"]]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

# Load features
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}')

Spange: (26, 13), DRFP filtered: (24, 122)


In [3]:
# Featurizer
class CombinedFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp])
    
    def featurize_torch(self, X, flip=False):
        return torch.tensor(self.featurize(X, flip))

print('Featurizer defined')

Featurizer defined


In [4]:
# MLP Model
class MLPModelInternal(nn.Module):
    def __init__(self, input_dim, hidden_dims=[32, 16], output_dim=3, dropout=0.05):
        super(MLPModelInternal, self).__init__()
        layers = [nn.BatchNorm1d(input_dim)]
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([nn.Linear(prev_dim, h_dim), nn.BatchNorm1d(h_dim), nn.ReLU(), nn.Dropout(dropout)])
            prev_dim = h_dim
        layers.extend([nn.Linear(prev_dim, output_dim), nn.Sigmoid()])
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

class MLPEnsemble:
    def __init__(self, hidden_dims=[32, 16], n_models=5, data='single'):
        self.hidden_dims = hidden_dims
        self.n_models = n_models
        self.data_type = data
        self.featurizer = CombinedFeaturizer(mixed=(data=='full'))
        self.models = []

    def train_model(self, X_train, y_train, epochs=200, batch_size=32, lr=5e-4):
        X_std = self.featurizer.featurize_torch(X_train, flip=False)
        y_vals = torch.tensor(y_train.values)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize_torch(X_train, flip=True)
            X_all = torch.cat([X_std, X_flip], dim=0)
            y_all = torch.cat([y_vals, y_vals], dim=0)
        else:
            X_all, y_all = X_std, y_vals
            
        input_dim = X_all.shape[1]
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        for i in range(self.n_models):
            torch.manual_seed(42 + i * 13)
            np.random.seed(42 + i * 13)
            model = MLPModelInternal(input_dim, hidden_dims=self.hidden_dims, dropout=0.05).to(device)
            model.train()
            self.models.append(model)
            
            dataset = TensorDataset(X_all, y_all)
            loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
            criterion = nn.HuberLoss()
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
            
            for epoch in range(epochs):
                epoch_loss = 0.0
                for inputs, targets in loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    optimizer.step()
                    epoch_loss += loss.item() * inputs.size(0)
                scheduler.step(epoch_loss / len(dataset))

    def predict(self, X):
        device = next(self.models[0].parameters()).device
        if self.data_type == 'full':
            X_std = self.featurizer.featurize_torch(X, flip=False).to(device)
            X_flip = self.featurizer.featurize_torch(X, flip=True).to(device)
            pred_sum = torch.zeros((len(X), 3)).to(device)
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    pred_sum += (model(X_std) + model(X_flip)) * 0.5
            return (pred_sum / self.n_models).cpu().numpy()
        else:
            X_std = self.featurizer.featurize_torch(X).to(device)
            pred_sum = torch.zeros((len(X), 3)).to(device)
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    pred_sum += model(X_std)
            return (pred_sum / self.n_models).cpu().numpy()

print('MLPEnsemble defined')

MLPEnsemble defined


In [5]:
# LightGBM Model
class LGBMModel:
    def __init__(self, data='single'):
        self.data_type = data
        self.featurizer = CombinedFeaturizer(mixed=(data=='full'))
        self.models = []
        self.params = {'objective': 'regression', 'metric': 'mse', 'boosting_type': 'gbdt',
                       'num_leaves': 31, 'learning_rate': 0.05, 'feature_fraction': 0.8,
                       'bagging_fraction': 0.8, 'bagging_freq': 5, 'verbose': -1, 'seed': 42}

    def train_model(self, X_train, y_train, num_boost_round=200):
        X_feat = self.featurizer.featurize(X_train)
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = np.vstack([X_feat, X_flip])
            y_all = np.vstack([y_train.values, y_train.values])
        else:
            X_all, y_all = X_feat, y_train.values
        
        self.models = []
        for i in range(3):
            train_data = lgb.Dataset(X_all, label=y_all[:, i])
            model = lgb.train(self.params, train_data, num_boost_round=num_boost_round)
            self.models.append(model)

    def predict(self, X):
        if self.data_type == 'full':
            X_std = self.featurizer.featurize(X, flip=False)
            X_flip = self.featurizer.featurize(X, flip=True)
            preds = np.zeros((len(X), 3))
            for i, model in enumerate(self.models):
                preds[:, i] = (model.predict(X_std) + model.predict(X_flip)) / 2
        else:
            X_feat = self.featurizer.featurize(X)
            preds = np.zeros((len(X), 3))
            for i, model in enumerate(self.models):
                preds[:, i] = model.predict(X_feat)
        return np.clip(preds, 0, 1)

print('LGBMModel defined')

LGBMModel defined


In [6]:
# Function to run CV with specific weights
def run_cv_with_weights(mlp_weight, lgbm_weight, verbose=True):
    """Run full CV with specified ensemble weights."""
    if verbose:
        print(f"\n=== Testing weights: MLP {mlp_weight}, LightGBM {lgbm_weight} ===")
    
    # Single solvent CV
    X, Y = load_data("single_solvent")
    all_preds_single = []
    all_actuals_single = []
    
    for fold_idx, split in enumerate(generate_leave_one_out_splits(X, Y)):
        (train_X, train_Y), (test_X, test_Y) = split
        
        # Train MLP
        mlp = MLPEnsemble(hidden_dims=[32, 16], n_models=5, data='single')
        mlp.train_model(train_X, train_Y, epochs=200)
        pred_mlp = mlp.predict(test_X)
        
        # Train LightGBM
        lgbm = LGBMModel(data='single')
        lgbm.train_model(train_X, train_Y, num_boost_round=200)
        pred_lgbm = lgbm.predict(test_X)
        
        # Weighted ensemble
        pred_ensemble = mlp_weight * pred_mlp + lgbm_weight * pred_lgbm
        pred_ensemble = np.clip(pred_ensemble, 0, 1)
        
        all_preds_single.append(pred_ensemble)
        all_actuals_single.append(test_Y.values)
        
        if verbose and (fold_idx + 1) % 8 == 0:
            print(f"  Single solvent: {fold_idx + 1}/24 folds done")
    
    preds_single = np.vstack(all_preds_single)
    actuals_single = np.vstack(all_actuals_single)
    mse_single = np.mean((actuals_single - preds_single) ** 2)
    
    # Full data CV
    X, Y = load_data("full")
    all_preds_full = []
    all_actuals_full = []
    
    for fold_idx, split in enumerate(generate_leave_one_ramp_out_splits(X, Y)):
        (train_X, train_Y), (test_X, test_Y) = split
        
        # Train MLP
        mlp = MLPEnsemble(hidden_dims=[32, 16], n_models=5, data='full')
        mlp.train_model(train_X, train_Y, epochs=200)
        pred_mlp = mlp.predict(test_X)
        
        # Train LightGBM
        lgbm = LGBMModel(data='full')
        lgbm.train_model(train_X, train_Y, num_boost_round=200)
        pred_lgbm = lgbm.predict(test_X)
        
        # Weighted ensemble
        pred_ensemble = mlp_weight * pred_mlp + lgbm_weight * pred_lgbm
        pred_ensemble = np.clip(pred_ensemble, 0, 1)
        
        all_preds_full.append(pred_ensemble)
        all_actuals_full.append(test_Y.values)
        
        if verbose and (fold_idx + 1) % 4 == 0:
            print(f"  Full data: {fold_idx + 1}/13 folds done")
    
    preds_full = np.vstack(all_preds_full)
    actuals_full = np.vstack(all_actuals_full)
    mse_full = np.mean((actuals_full - preds_full) ** 2)
    
    # Overall MSE
    n_single = len(actuals_single)
    n_full = len(actuals_full)
    overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)
    
    if verbose:
        print(f"\nResults for MLP {mlp_weight}, LightGBM {lgbm_weight}:")
        print(f"  Single Solvent MSE: {mse_single:.6f}")
        print(f"  Full Data MSE: {mse_full:.6f}")
        print(f"  Overall MSE: {overall_mse:.6f}")
    
    return overall_mse, mse_single, mse_full

print('CV function defined')

CV function defined


In [7]:
# Test weight: MLP 0.7, LightGBM 0.3 (more MLP emphasis)
results = {}
mse_07_03, mse_single_07, mse_full_07 = run_cv_with_weights(0.7, 0.3)
results['0.7/0.3'] = mse_07_03


=== Testing weights: MLP 0.7, LightGBM 0.3 ===


  Single solvent: 8/24 folds done


  Single solvent: 16/24 folds done


  Single solvent: 24/24 folds done


  Full data: 4/13 folds done


  Full data: 8/13 folds done


  Full data: 12/13 folds done



Results for MLP 0.7, LightGBM 0.3:
  Single Solvent MSE: 0.009346
  Full Data MSE: 0.008833
  Overall MSE: 0.009012


In [8]:
# Compare with baseline (0.6/0.4)
print("\n=== WEIGHT COMPARISON ===")
print(f"Baseline (0.6/0.4): CV 0.009004 (from exp_013)")
print(f"MLP 0.7, LightGBM 0.3: CV {mse_07_03:.6f}")

if mse_07_03 < 0.009004:
    improvement = (0.009004 - mse_07_03) / 0.009004 * 100
    print(f"\n✓ IMPROVEMENT: {improvement:.2f}% better than baseline!")
else:
    degradation = (mse_07_03 - 0.009004) / 0.009004 * 100
    print(f"\n✗ WORSE: {degradation:.2f}% worse than baseline")

print(f"\nexp_007 [32,16] alone: CV 0.009262, LB 0.0932 (BEST LB)")
print(f"exp_013 ensemble (0.6/0.4): CV 0.009004")


=== WEIGHT COMPARISON ===
Baseline (0.6/0.4): CV 0.009004 (from exp_013)
MLP 0.7, LightGBM 0.3: CV 0.009012

✗ WORSE: 0.09% worse than baseline

exp_007 [32,16] alone: CV 0.009262, LB 0.0932 (BEST LB)
exp_013 ensemble (0.6/0.4): CV 0.009004


In [9]:
# Save best submission if this is better
if mse_07_03 < 0.009004:
    print("\nThis weight combination is better! Saving submission...")
    # Would need to re-run with proper submission format
else:
    print("\nKeeping exp_013 (0.6/0.4) as best ensemble.")
    print("The 0.6/0.4 weighting appears optimal.")

print("\n=== SUMMARY ===")
print(f"Best ensemble: MLP 0.6, LightGBM 0.4 (CV 0.009004)")
print(f"Best LB model: [32,16] MLP alone (CV 0.009262, LB 0.0932)")
print(f"\nRecommendation: Submit exp_013 ensemble when submissions reset.")


Keeping exp_013 (0.6/0.4) as best ensemble.
The 0.6/0.4 weighting appears optimal.

=== SUMMARY ===
Best ensemble: MLP 0.6, LightGBM 0.4 (CV 0.009004)
Best LB model: [32,16] MLP alone (CV 0.009262, LB 0.0932)

Recommendation: Submit exp_013 ensemble when submissions reset.
